In [ ]:
folderPath = "D:/"

In [ ]:
import numpy as np
import pandas as pd

def PrintGreen(text):
    print('\x1b[6;30;42m' + text + '\x1b[0m')
    
def PrintRed(text):
    print('\33[41m' + text + '\x1b[0m')

def LoadData(filename, rowsName, columnsName):
    newDataframe = pd.read_csv(filename, na_values = 'null')
    if newDataframe.shape[0] > 0 and newDataframe.shape[1] > 0:
        PrintGreen("Loading " + filename + " succeeded");
    else:
        PrintRed("Loading " + filename + " failed!");

    print(rowsName + " = " + str(newDataframe.shape[0]))
    print(columnsName + " = " + str(newDataframe.shape[1]))

    return newDataframe

# Ensure to show all columns
pd.set_option('display.max_columns', None)

In [ ]:
dataPosRotModelSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_ModelSpace_60Hz_Training.csv', "Frames", "PoseComponents")
dataPosRotModelSpace.head(15)

In [ ]:
dataPosRotModelSpaceTesting = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_ModelSpace_60Hz_Testing.csv', "Frames", "PoseComponents")
dataPosRotModelSpaceTesting.head(15)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

posRotModelSpaceDataScaler = preprocessing.StandardScaler()
dataPosRotModelSpaceScaled = posRotModelSpaceDataScaler.fit_transform(dataPosRotModelSpace.values)

posRotModelSpaceDataScalerTesting = preprocessing.StandardScaler()
posRotModelSpaceDataScalerTesting.fit(dataPosRotModelSpace.values)
dataPosRotModelSpaceScaledTesting = posRotModelSpaceDataScalerTesting.transform(dataPosRotModelSpaceTesting.values)

# Input (Pose + Features)
data_input = dataPosRotModelSpaceScaled
print(data_input.shape)

# Output (Next pose)
data_output = dataPosRotModelSpaceScaled
print(data_output.shape)

In [ ]:
# Tensorboard
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

logdir = folderPath + "TensorBoard/" # + datetime.now().strftime("%Y%m%d") + "/"
tensorBoardWriter = SummaryWriter(logdir)

In [ ]:
import torch
import torch.nn as nn

# Set fixed random number seed
torch.manual_seed(42)

def ConstructModelEncoder(inputFeatures, outputFeatures):
    hidden_layer1 = 936
    hidden_layer2 = 468
    hidden_layer3 = 234
    hidden_layer4 = 117
    hidden_layer5 = 78
    hidden_layer6 = 39

    hidden_layer_v2_1 = 624
    hidden_layer_v2_2 = 312
    hidden_layer_v2_3 = 156
    hidden_layer_v2_4 = 78
    model = nn.Sequential(nn.Linear(inputFeatures, hidden_layer4),
                          nn.ReLU()
                          )
    
    def init_weights(model):
        if isinstance(model, nn.Linear):
            torch.nn.init.xavier_uniform(model.weight)
            model.bias.data.fill_(0.01)

    init_weights(model)
    print(model)
    return model

def ConstructModelDecoder(inputFeatures, outputFeatures):
    hidden_layer1 = 936
    hidden_layer2 = 468
    hidden_layer3 = 234
    hidden_layer4 = 117
    hidden_layer5 = 78
    hidden_layer6 = 39

    hidden_layer_v2_1 = 624
    hidden_layer_v2_2 = 312
    hidden_layer_v2_3 = 156
    hidden_layer_v2_4 = 78
    model = nn.Sequential(nn.Linear(hidden_layer4, outputFeatures))
    
    def init_weights(model):
        if isinstance(model, nn.Linear):
            torch.nn.init.xavier_uniform(model.weight)
            model.bias.data.fill_(0.01)

    init_weights(model)
    print(model)
    return model

In [ ]:
mini_batch_size = 128
epochs = 100
lr = 1e-4

In [ ]:
from random import shuffle
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

# Convert an input and output tensor into a dataset
# Inputs:
# X: torch.tensor specifying the training input data
# y: torch.tensor specifying the training labels
class CreatePytorchDataset(Dataset):
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]
    def __init__(self,X, y):
        self.x_train=X
        self.y_train=y 
    def __len__(self):
        return len(self.y_train)

# Convert Pandas DataFrames to PyTorch tensors
X = torch.tensor(data_input, dtype=torch.float32)
y = torch.tensor(data_output, dtype=torch.float32)

# Convert Pandas DataFrames to PyTorch tensors
Xt = torch.tensor(dataPosRotModelSpaceScaledTesting, dtype=torch.float32)
yt = torch.tensor(dataPosRotModelSpaceScaledTesting, dtype=torch.float32)

# Construct the PyTorch dataset and data loaders
torchDataset = CreatePytorchDataset(X, y)
train_loader = torch.utils.data.DataLoader(torchDataset, batch_size=mini_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torchDataset, batch_size=mini_batch_size, shuffle=True)

# Construct the model and optimizer and train the model
inputFeatures = X.shape[1]
outputFeatures = y.shape[1]
modelEncoder = ConstructModelEncoder(inputFeatures, 117)
modelDecoder = ConstructModelDecoder(117, outputFeatures)

#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(list(modelEncoder.parameters()) + list(modelDecoder.parameters()), lr=lr)

# mean-squared error loss
criterion = nn.MSELoss()

In [ ]:
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

def ExportModelToOnnx(model, filename, dummy_input):
    torch.onnx.export(model,                     # model being run
                      dummy_input,               # model input (or a tuple for multiple inputs)
                      filename,                  # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

In [ ]:
def Training(modelEncoder, modelDecoder, train_loader, test_loader, optimizer):

    timeCode = datetime.now().strftime("%H:%M:%S") # used for TensorBoard

    # Re-train on the same data for the given amount of epochs
    losses = []
    lossesTesting = []
    for epoch in range(0, epochs):

        print("Epoch " + str(epoch) + ": ")
        running_loss = 0.0
        current_mini_batch = 0

        # Mini-batches
        for i, data in enumerate(train_loader, 0):

            # Get the inputs and labels/outputs from the dataset
            X, y = data

            #################
            # Backpropagation

            # Zero the gradients
            modelEncoder.zero_grad()
            modelDecoder.zero_grad()

            # Perform forward pass and compute prediction
            pred_y = modelDecoder(modelEncoder(X))

            # Compute loss
            loss = criterion(pred_y, y)
            running_loss += loss.item()
            tensorBoardWriter.add_scalar(timeCode + " Loss / train", loss.item(), current_mini_batch)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            print_each = 100
            if current_mini_batch % print_each == print_each - 1:
                averaged_loss = running_loss / print_each
                print("Loss after mini-batch %5d: %.5f" % (current_mini_batch + 1, averaged_loss))
                tensorBoardWriter.add_scalar(timeCode + " Running loss / train", averaged_loss, current_mini_batch)
                losses.append(averaged_loss)
                running_loss = 0.0

            current_mini_batch = current_mini_batch + 1

        pred_y_testing = modelDecoder(modelEncoder(Xt))
        total = 0.0
        for i in range(len(pred_y_testing)):
            loss_testing = criterion(pred_y_testing[i], yt[i])
            total += loss_testing.item()
        lossesTesting.append(total)
        print(total)

    lossData = pd.DataFrame(losses)
    lossData.to_csv("D:/MMDatabaseAutoEncoderModelSpace1layer1872to117SplitModelOpset11LR1e-4Epoch100Batch128_WithValidationTest.csv", index=False)
    plt.plot(losses)
    plt.ylabel('loss')
    plt.xlabel('mini-batch')
    plt.title("Learning rate %f"%(lr))
    plt.show()

    lossDataTesting = pd.DataFrame(lossesTesting)
    lossDataTesting.to_csv("D:/MMDatabaseAutoEncoderModelSpace1layer1872to117SplitModelOpset11LR1e-4Epoch100Batch128_WithValidationTest_This.csv", index=False)
    plt.plot(lossesTesting)
    plt.ylabel('loss')
    plt.xlabel('mini-batch')
    plt.title("Learning rate %f"%(lr))
    plt.show()

    # TensorBoard
    tensorBoardWriter.close()

Training(modelEncoder, modelDecoder, train_loader, test_loader, optimizer)

# Export the trained model to an ONNX file
filenameEncoder = folderPath + 'MMDatabaseAutoEncoderModelSpace1layer1872to117SplitModelOpset11LR1e-4Epoch100Batch128_WithValidationTest.onnx'
ExportModelToOnnx(modelEncoder, filenameEncoder, X)

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("D:/MMDatabaseAutoEncoderModelSpace1layer1872to117SplitModelOpset11LR1e-4Epoch100Batch128_WithValidationTest.onnx")
X = torch.tensor(dataPosRotModelSpaceScaledTesting, dtype=torch.float32)
ort_input = {"input": X.numpy()}
ort_outs = ort_session.run(None, ort_input)

In [ ]:
finalFeatures = pd.DataFrame(ort_outs[0])
finalFeatures

In [ ]:
finalFeatures.to_csv("D:/InferencedFeatures_MMDatabaseAutoEncoderModelSpace1layer1872to117SplitModelOpset11LR1e-4Epoch100Batch128_WithValidationTest_Testing.csv", index=False)